<a href="https://colab.research.google.com/github/lenabic/bsd23_bichler_lena/blob/main/Netflix_Titles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My Dataset

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap

In [2]:
df = pd.read_csv("netflix_titles.csv")

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


# Data Cleaning

## Drop Duplicates

In [4]:
# see the sum before
df.shape

(8807, 12)

In [5]:
dt = df.drop_duplicates()

In [6]:
# sum after dropping duplicates
dt.shape

(8807, 12)

after checking if there are any duplicates before deleting them from df, you can see that there are no identical duplicates so this is clean

## Consistency



---

**Consistency**

check the columns to make sure the values of them are consistent

### Check Type for Consistency

In [7]:
df.type.value_counts()

,count
type,
Movie,6131
TV Show,2676


type only uses two values - movie and tv show. this column is "clean".

### Check rating for Consistency

In [8]:
# list all the different rating types used in this data set
df.rating.value_counts()

,count
rating,
TV-MA,3207
TV-14,2160
TV-PG,863
R,799
PG-13,490
TV-Y7,334
TV-Y,307
PG,287
TV-G,220





---


remove the obvious fehlerhaften werte - minute angabn


some rating types seem obviousöy wrong (minute types - 74 min 84 min 66 min). There might have been a mix up in what column these belong in. Regardless, these rows are faulty and will be removed.

In [9]:
# move all invalid ratings into a new data set
invalid_ratings = df[df['rating'].str.contains(r'^\d+ min$', na=False)]

In [10]:
invalid_ratings

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
5541,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,74 min,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,84 min,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,"August 15, 2016",2015,66 min,NaN,Movies,The comic puts his trademark hilarious/thought...


In [11]:
# remove the invalid ratings from df
df = df.drop(invalid_ratings.index)

In [12]:
# check to see if it was done correctly
df.rating.value_counts()

,count
rating,
TV-MA,3207
TV-14,2160
TV-PG,863
R,799
PG-13,490
TV-Y7,334
TV-Y,307
PG,287
TV-G,220


The three rows have now been removed



---



New rating types

Because the other types of ratings all use different rating systems that mean somewhat the same, I have decided to combine the ratings that fit together to make it more überschaubar. This is because:


*   less rating types make it easier to analyse
*   the rating types mean the same anyway for different age groups, they just use a different method to describe it



In [13]:
# rename all TV-MA, R and NC-17 to Adult
df['rating'] = df['rating'].replace(['TV-MA', 'R', 'NC-17'], 'Adult')

In [14]:
# rename all TV-14 and PG-13 to Teen/Young Adult
df['rating'] = df['rating'].replace(['TV-14', 'PG-13'], 'Teen/Young Adult')

In [15]:
# rename all TV-PG, PG, TV-G, G to Family Friendly
df['rating'] = df['rating'].replace(['TV-PG', 'PG', 'TV-G', 'G'], 'Family Friendly')

In [16]:
# rename all TV-Y7, TV-Y, TV-Y7-FV to Kids
df['rating'] = df['rating'].replace(['TV-Y7', 'TV-Y', 'TV-Y7-FV'], 'Kids')

In [17]:
# rename all NR and UR to Unrated
df['rating'] = df['rating'].replace(['NR', 'UR'], 'Unrated')

In [18]:
# check if done correctly
df.rating.value_counts()

,count
rating,
Adult,4009
Teen/Young Adult,2650
Family Friendly,1411
Kids,647
Unrated,83


all rating types have now been made easier and visually better

### Check duration for Consistency

In [19]:
df.duration.value_counts()

,count
duration,
1 Season,1793
2 Seasons,425
3 Seasons,199
90 min,152
94 min,146
...,...
16 min,1
186 min,1
193 min,1


This uses two types of angabe for duration - one in number of seasons and one in minutes.
 I have decided to convert the season times into the standard length of a season in minutes.

 Google search and asking ai have ergeben, that taken into account the different country types of different durations, a 10 episode season will be used for this calculation as the mean per season, as it would take me too much time to look up each show individually to check how many episodes and minutes there are.

 we describe here, taht an epsiode takes 45 minutes on average

In [20]:
# replace 1 Season with 450 min
df['duration'] = df['duration'].replace('1 Season', '450 min')

In [21]:
df_seasons = df[df['duration'].str.contains('Seasons', na=False)]

In [22]:
df_seasons['num_seasons'] = df_seasons['duration'].str.extract('(\d+)').astype(int)

<ipython-input-22-8df2e30d85c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
df_seasons['duration_in_minutes'] = df_seasons['num_seasons'] * 450

<ipython-input-23-d373ca889af0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
df_seasons['duration'] = df_seasons['duration_in_minutes'].astype(str) + ' min'

<ipython-input-24-8455989ad93d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
df.update(df_seasons[['title', 'duration']])

In [26]:
df.duration.value_counts()

,count
duration,
450 min,1793
900 min,425
1350 min,199
90 min,152
94 min,146
...,...
16 min,1
186 min,1
193 min,1


## Drop Not Needed Columns

There are some columns that are not needed for my evaluation/ analyzation of this data set - show_id, director, cast nd description.

Let's remove them!

In [27]:
df = df.drop(columns='show_id', axis=1)

In [28]:
df = df.drop(columns='director', axis=1)

In [29]:
df = df.drop(columns='cast', axis=1)

In [30]:
df = df.drop(columns='description', axis=1)

In [31]:
df.head()

,type,title,country,date_added,release_year,rating,duration,listed_in
0,Movie,Dick Johnson Is Dead,United States,"September 25, 2021",2020,Teen/Young Adult,90 min,Documentaries
1,TV Show,Blood & Water,South Africa,"September 24, 2021",2021,Adult,900 min,"International TV Shows, TV Dramas, TV Mysteries"
2,TV Show,Ganglands,NaN,"September 24, 2021",2021,Adult,450 min,"Crime TV Shows, International TV Shows, TV Act..."
3,TV Show,Jailbirds New Orleans,NaN,"September 24, 2021",2021,Adult,450 min,"Docuseries, Reality TV"
4,TV Show,Kota Factory,India,"September 24, 2021",2021,Adult,900 min,"International TV Shows, Romantic TV Shows, TV ..."


## Missing Variables - NaN

In [32]:
# cheking how many NaN there are in each column
df.isnull().sum()

,0
type,0
title,0
country,831
date_added,10
release_year,0
rating,4
duration,0
listed_in,0


### NaN in Ratings

In [33]:
# check ratings NaN
df[df['rating'].isnull()]

,type,title,country,date_added,release_year,rating,duration,listed_in
5989,Movie,13TH: A Conversation with Oprah Winfrey & Ava ...,NaN,"January 26, 2017",2017,NaN,37 min,Movies
6827,TV Show,Gargantia on the Verdurous Planet,Japan,"December 1, 2016",2013,NaN,450 min,"Anime Series, International TV Shows"
7312,TV Show,Little Lunch,Australia,"February 1, 2018",2015,NaN,450 min,"Kids' TV, TV Comedies"
7537,Movie,My Honor Was Loyalty,Italy,"March 1, 2017",2015,NaN,115 min,Dramas


as I am taking the NaN as the same thing as UNrated, I will simply add them to the Unrated type.

In [34]:
# rewrite all NaN to Unrated
dt = df['rating'] = df['rating'].fillna('Unrated')

### NaN in Country

In [35]:
# list the NaN in country
df[df['country'].isnull()]

,type,title,country,date_added,release_year,rating,duration,listed_in
2,TV Show,Ganglands,NaN,"September 24, 2021",2021,Adult,450 min,"Crime TV Shows, International TV Shows, TV Act..."
3,TV Show,Jailbirds New Orleans,NaN,"September 24, 2021",2021,Adult,450 min,"Docuseries, Reality TV"
5,TV Show,Midnight Mass,NaN,"September 24, 2021",2021,Adult,450 min,"TV Dramas, TV Horror, TV Mysteries"
6,Movie,My Little Pony: A New Generation,NaN,"September 24, 2021",2021,Family Friendly,91 min,Children & Family Movies
10,TV Show,"Vendetta: Truth, Lies and The Mafia",NaN,"September 24, 2021",2021,Adult,450 min,"Crime TV Shows, Docuseries, International TV S..."
...,...,...,...,...,...,...,...,...
8718,Movie,Westside vs. the World,NaN,"August 9, 2019",2019,Adult,96 min,"Documentaries, Sports Movies"
8759,Movie,World's Weirdest Homes,NaN,"February 1, 2019",2015,Family Friendly,49 min,Movies
8783,TV Show,Yoko,NaN,"June 23, 2018",2016,Kids,450 min,Kids' TV
8785,TV Show,YOM,NaN,"June 7, 2018",2016,Kids,450 min,Kids' TV


Due to the fact that I cannot simply assume that with 831 productions they will just be in the majority of the countries, I will add a new section "Not specified"

In [36]:
# put all NaN as a new country type "Not Specified"
df['country'] = df['country'].fillna('Not Specified')

In [37]:
df.country.isnull().sum()

0

In [38]:
# value counts for country
df.country.value_counts()

,count
country,
United States,2815
India,972
Not Specified,831
United Kingdom,419
Japan,245
...,...
"Romania, Bulgaria, Hungary",1
"Uruguay, Guatemala",1
"France, Senegal, Belgium",1


In some cases (e.g.: Country) it is useful for my further analysis to split it into each country using the explode method

In [39]:
# explode method for country
dt['country'] =  df['country'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)

### NaN in date_added

In [40]:
df[df['date_added'].isnull()]

,type,title,country,date_added,release_year,rating,duration,listed_in
6066,TV Show,A Young Doctor's Notebook and Other Stories,United Kingdom,NaN,2013,Adult,900 min,"British TV Shows, TV Comedies, TV Dramas"
6174,TV Show,Anthony Bourdain: Parts Unknown,United States,NaN,2018,Family Friendly,2250 min,Docuseries
6795,TV Show,Frasier,United States,NaN,2003,Family Friendly,4950 min,"Classic & Cult TV, TV Comedies"
6806,TV Show,Friends,United States,NaN,2003,Teen/Young Adult,4500 min,"Classic & Cult TV, TV Comedies"
6901,TV Show,Gunslinger Girl,Japan,NaN,2008,Teen/Young Adult,900 min,"Anime Series, Crime TV Shows"
7196,TV Show,Kikoriki,Not Specified,NaN,2010,Kids,900 min,Kids' TV
7254,TV Show,La Familia P. Luche,United States,NaN,2012,Teen/Young Adult,1350 min,"International TV Shows, Spanish-Language TV Sh..."
7406,TV Show,Maron,United States,NaN,2016,Adult,1800 min,TV Comedies
7847,TV Show,Red vs. Blue,United States,NaN,2015,Unrated,5850 min,"TV Action & Adventure, TV Comedies, TV Sci-Fi ..."
8182,TV Show,The Adventures of Figaro Pho,Australia,NaN,2015,Kids,900 min,"Kids' TV, TV Comedies"


Because it is a small amount, I have decided to just delete the NaN date_added

In [51]:
# delete all rows with NaN in date_added
df = df.dropna(subset=['date_added'])

In [52]:
df.isnull().sum()

,0
type,0
title,0
country,0
date_added,0
release_year,0
rating,0
duration,0
listed_in,0
primary_genre,0
primary_country,0


## Shortening

In order to get a more accurate result when doing the Data Modelling, I will be shortening the columns that have more than one thing mentioned in a row (listed_in and country). I assume that the most important country and genre is always the one mentioned first, so I will be shortening the INhalt to just that information.

In [53]:
df['primary_genre'] = df['listed_in'].str.split(',').str[0]

<ipython-input-53-035f66a24cd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [54]:
df['primary_country'] = df['country'].str.split(',').str[0]

<ipython-input-54-31d459d9ee9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [55]:
df.primary_genre.value_counts()

,count
primary_genre,
Dramas,1600
Comedies,1210
Action & Adventure,859
Documentaries,829
International TV Shows,773
Children & Family Movies,605
Crime TV Shows,399
Kids' TV,386
Stand-Up Comedy,334


In [56]:
df.primary_country.value_counts()

,count
primary_country,
United States,3202
India,1008
Not Specified,830
United Kingdom,627
Canada,271
...,...
Namibia,1
Senegal,1
Luxembourg,1


In [57]:
df.head()

,type,title,country,date_added,release_year,rating,duration,listed_in,primary_genre,primary_country
0,Movie,Dick Johnson Is Dead,United States,"September 25, 2021",2020,Teen/Young Adult,90 min,Documentaries,Documentaries,United States
1,TV Show,Blood & Water,South Africa,"September 24, 2021",2021,Adult,900 min,"International TV Shows, TV Dramas, TV Mysteries",International TV Shows,South Africa
2,TV Show,Ganglands,Not Specified,"September 24, 2021",2021,Adult,450 min,"Crime TV Shows, International TV Shows, TV Act...",Crime TV Shows,Not Specified
3,TV Show,Jailbirds New Orleans,Not Specified,"September 24, 2021",2021,Adult,450 min,"Docuseries, Reality TV",Docuseries,Not Specified
4,TV Show,Kota Factory,India,"September 24, 2021",2021,Adult,900 min,"International TV Shows, Romantic TV Shows, TV ...",International TV Shows,India


I have now created two columns more, to more accurately analyse the listed_in and country columns

# Data Modelling
Analyzing the Data based on 10 thought out questions

## Which Genres are most commomly used?

In [60]:
df.primary_genre.value_counts().head()

,count
primary_genre,
Dramas,1600
Comedies,1210
Action & Adventure,859
Documentaries,829
International TV Shows,773


The genres most commonly used are Dramas, Comedies, Action & Adventures, closely followed by Documentaries and International TV Shows.

## Are there more Movies or more TV Shows in this list from Netflix repertoire?

In [58]:
# more movies or tv shows
df.type.value_counts()

,count
type,
Movie,6128
TV Show,2666


There are more movies listed in this data set

## Gibt es Muster in der Häufigkeit von Produktionen basierend auf dem Monat des Hinzufügens (date added)?

## Which countries have the most productions?

In [61]:
count = df.primary_country.value_counts()

In [66]:
top_10_countries = count.head(10).reset_index()
top_10_countries.columns = ['country', 'count']

# Visualisierung mit Plotly
fig = px.bar(
    top_10_countries,
    x='count',
    y='country',
    orientation='h',
    title='Top 10 Countries with the most Netflix Productions',
    labels={'count': 'Number of Productions', 'country': 'Country'},
    color='count',
   )
fig.update_layout(yaxis=dict(categoryorder='total ascending'))
fig.show()

The top 3 countries with the most productions are United States, India and United Kingdom. As seen in the plot, United States clearly is the winner, having 2194 productions more than second place, India.

## Most Productions according to type?